This can create multiple topic model files according to types and date ranges.  We begin with the same basic dataframe we have been using, clean it up a bit, and then set the parameters and run the routine.  It could take a long time.

In [ ]:
import sys
import json
import datetime
import pickle
import gc
import time

from operator import itemgetter
from collections import defaultdict

import pandas as pd
import numpy as np

import gensim
from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models, similarities, matutils
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import pprint
pp = pprint.PrettyPrinter(indent=4)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
with open('clean_data_full.pickle', 'rb') as file:
    art = pickle.load(file)
# List of Extra Stop words
stopwords = [
    'week','another','thing','month','day','come',
    'york','away','left','wrote','came','tell','asked',
    'left','right','hand','point','often','talk','head','point','ago','whether',
    'hour','group','became','become','becomes','often','sometimes','usually','page','sequence','doi','p'
    ]
# Remove Extra Stop Words
art['article_text_nostop_extra'] = [[word for word in x if not word in stopwords and len(word)>2] for x in art['tokenized_nopunc_lower_nostop_extra_lemmatized']]


In [ ]:
art['string'] = [' '.join(x) for x in art['article_text_nostop_extra']]

In [ ]:
#It is easy enough to add another atype, 'research-article', and create another loop to run that along with the book-review articles.
#We did not do that because we used different numbers of topics for each of these atypes.
atype='book-review'
years = [ #range (1982,1985),
          range(1985,1990),
          range(1990,1995),
          range(1995,2000), 
          range(2000,2005), 
          range(2005,2010),
          range(2010,2015)
        ]
art=art[art['type']==atype]

In [ ]:
art.head()

In [ ]:
dict_list=[]

for i in years:
    newart=art.copy()
    newart.astype({'year':'int'}).dtypes
    x=int(i[0])-1
    y=int(i[-1])
    newart=newart[newart['year']<y]
    newart=newart[newart['year']>x]
    texts=newart['article_text_nostop_extra'].tolist()
    for a in range(len(texts)):
        for b in range(len (texts[a])):
            dict_list.append(texts[a][b])
    
    res = [sub.split() for sub in dict_list] 
    
    dictionary = corpora.Dictionary(res)
    corpus = [dictionary.doc2bow(text) for text in texts]
   
    corpora.MmCorpus.serialize('corpus_allbooks'+str(i[0])+'.mm', corpus)
    no_of_topics = 17
    ldamodel17 = LdaModel(corpus, num_topics=no_of_topics, id2word=dictionary, passes=50, alpha='auto', eval_every=2000)
    # Save Model (24)
    ldamodel17.save('l2a_17t_50p_autoalpha_books_all_{}_val.model'.format(i[0]))
    vis = pyLDAvis.gensim.prepare(ldamodel17, corpus, dictionary)
    pyLDAvis.save_html(vis,'visualization_research_all'+str(i[0])+'.html')

    
    

In [ ]:
    for a in range(len(texts)):
        for b in range(len (texts[a])):
            dict_list.append(texts[a][b])


    res = [sub.split() for sub in dict_list] 
    dictionary = corpora.Dictionary(res)
    
    corpus = [dictionary.doc2bow(text) for text in texts]
    corpora.MmCorpus.serialize('corpus_book'+str(i[0])+'.mm', corpus)
    
#     vec = CountVectorizer(max_df =.95,min_df = 10,stop_words='english')
#     counts = vec.fit_transform(texts).transpose()
#     corpus = matutils.Sparse2Corpus(counts)
#     with open('full_set_countvec_corpus.'+str(i[0])+'.pickle','wb') as file:
#         pickle.dump(corpus,file)
    
    no_of_topics = 17
    ldamodel17 = LdaModel(corpus, num_topics=no_of_topics, id2word=dictionary, passes=50, alpha='auto', eval_every=2000)
    # Save Model (24)
    ldamodel17.save('lda_17t_50p_autoalpha_book_{}_val.model'.format(i[0]))
    vis = pyLDAvis.gensim.prepare(ldamodel17, corpus, dictionary)
    pyLDAvis.save_html(vis,'visualization_book'+str(i[0])+'.html')
    
    